In [1]:
#import all libraries needed
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from citipy import citipy
import hvplot.pandas
from api_keys_mm import weather_api_key
from api_keys_mm import geoapify_key

In [4]:
#read cleaned data from CSV file in folder
clean_data = pd.read_csv(("drug_deaths_clean.csv"))
pd.set_option('display.max_columns', None)
clean_data.info()
clean_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3425 entries, 0 to 3424
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3425 non-null   int64  
 1   ID                  3425 non-null   object 
 2   Date                3425 non-null   object 
 3   DateType            3425 non-null   float64
 4   Age                 3425 non-null   float64
 5   Sex                 3422 non-null   object 
 6   Race                3419 non-null   object 
 7   ResidenceCity       3425 non-null   object 
 8   ResidenceCounty     3383 non-null   object 
 9   ResidenceState      3425 non-null   object 
 10  DeathCity           3425 non-null   object 
 11  InjuryState         1318 non-null   object 
 12  Heroin              3425 non-null   int64  
 13  Cocaine             3425 non-null   int64  
 14  Fentanyl            3425 non-null   object 
 15  Fentanyl_Analogue   3425 non-null   float64
 16  Oxycod

,Unnamed: 0,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,InjuryState,Heroin,Cocaine,Fentanyl,Fentanyl_Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,DeathCityGeo
0,2,16-0165,03/13/2016 12:00:00 AM,0.0,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,NaN,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,"Danbury, CT\n(41.393666, -73.451539)"
1,13,15-0232,05/14/2015 12:00:00 AM,1.0,50.0,Male,White,DANBURY,FAIRFIELD,CT,DANBURY,NaN,1,0,0,0.0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,"DANBURY, CT\n(41.393666, -73.451539)"
2,14,16-0028,01/13/2016 12:00:00 AM,0.0,29.0,Male,Black,NEW HAVEN,NEW HAVEN,CT,NEW HAVEN,NaN,1,0,0,0.0,0,0,0,0,1,0,1,0,0,0,NaN,0,1,"NEW HAVEN, CT\n(41.308252, -72.924161)"
3,18,16-0065,01/30/2016 12:00:00 AM,0.0,54.0,Male,White,MIDDLETOWN,MIDDLESEX,CT,MIDDLETOWN,NaN,1,0,0,0.0,0,0,1,0,0,0,0,0,0,0,NaN,0,1,"MIDDLETOWN, CT\n(41.544654, -72.651713)"
4,19,16-0889,12/20/2016 12:00:00 AM,0.0,32.0,Male,White,WINDHAM,WINDHAM,CT,WINDHAM,NaN,1,0,1,0.0,0,0,0,0,1,0,0,0,0,0,NaN,0,1,"Windham, CT\n(41.699744, -72.157703)"


In [6]:
#I noticed in code below when I tried to combine drug types, Fentanyl column is a string. See output below for why
#Turns out several drug types are strings. See df.info() above for which ones
clean_data["Fentanyl"].value_counts()
clean_data["Morphine_NotHeroin"].value_counts()

0                   3387
1                     34
1ES                    1
STOLE MEDS             1
NO RX BUT STRAWS       1
PCP NEG                1
Name: Morphine_NotHeroin, dtype: int64

In [7]:
#replace weird value with 1 so I cann add columns together
clean_data["Fentanyl"] = clean_data["Fentanyl"].replace(["1-A"],"1")
clean_data["Morphine_NotHeroin"] = clean_data["Morphine_NotHeroin"].replace(["1ES","STOLE MEDS", "NO RX BUT STRAWS","PCP NEG"],0)


clean_data["Fentanyl"].value_counts()
clean_data["Morphine_NotHeroin"].value_counts()

0    3387
1      34
0       4
Name: Morphine_NotHeroin, dtype: int64

In [8]:
#reformat all changed string columns as integers
clean_data["Fentanyl"] = clean_data["Fentanyl"].astype(int)
clean_data["Morphine_NotHeroin"] = clean_data["Morphine_NotHeroin"].astype(int)
clean_data["Fentanyl_Analogue"] = clean_data["Fentanyl_Analogue"].astype(int)
clean_data.head()

,Unnamed: 0,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,InjuryState,Heroin,Cocaine,Fentanyl,Fentanyl_Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,DeathCityGeo
0,2,16-0165,03/13/2016 12:00:00 AM,0.0,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,NaN,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,"Danbury, CT\n(41.393666, -73.451539)"
1,13,15-0232,05/14/2015 12:00:00 AM,1.0,50.0,Male,White,DANBURY,FAIRFIELD,CT,DANBURY,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,"DANBURY, CT\n(41.393666, -73.451539)"
2,14,16-0028,01/13/2016 12:00:00 AM,0.0,29.0,Male,Black,NEW HAVEN,NEW HAVEN,CT,NEW HAVEN,NaN,1,0,0,0,0,0,0,0,1,0,1,0,0,0,NaN,0,1,"NEW HAVEN, CT\n(41.308252, -72.924161)"
3,18,16-0065,01/30/2016 12:00:00 AM,0.0,54.0,Male,White,MIDDLETOWN,MIDDLESEX,CT,MIDDLETOWN,NaN,1,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,0,1,"MIDDLETOWN, CT\n(41.544654, -72.651713)"
4,19,16-0889,12/20/2016 12:00:00 AM,0.0,32.0,Male,White,WINDHAM,WINDHAM,CT,WINDHAM,NaN,1,0,1,0,0,0,0,0,1,0,0,0,0,0,NaN,0,1,"Windham, CT\n(41.699744, -72.157703)"


In [9]:
clean_data["Total Drug Deaths"] = clean_data["Heroin"] + clean_data["Cocaine"] + clean_data["Fentanyl"] + clean_data["Fentanyl_Analogue"] + clean_data["Oxycodone"] + clean_data["Oxymorphone"] + clean_data["Ethanol"] + clean_data["Hydrocodone"] + clean_data["Benzodiazepine"] + clean_data["Methadone"] + clean_data["Amphet"] + clean_data["Tramad"] + clean_data["Morphine_NotHeroin"]
clean_data.head()

,Unnamed: 0,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,InjuryState,Heroin,Cocaine,Fentanyl,Fentanyl_Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,DeathCityGeo,Total Drug Deaths
0,2,16-0165,03/13/2016 12:00:00 AM,0.0,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,NaN,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,"Danbury, CT\n(41.393666, -73.451539)",2
1,13,15-0232,05/14/2015 12:00:00 AM,1.0,50.0,Male,White,DANBURY,FAIRFIELD,CT,DANBURY,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,"DANBURY, CT\n(41.393666, -73.451539)",1
2,14,16-0028,01/13/2016 12:00:00 AM,0.0,29.0,Male,Black,NEW HAVEN,NEW HAVEN,CT,NEW HAVEN,NaN,1,0,0,0,0,0,0,0,1,0,1,0,0,0,NaN,0,1,"NEW HAVEN, CT\n(41.308252, -72.924161)",3
3,18,16-0065,01/30/2016 12:00:00 AM,0.0,54.0,Male,White,MIDDLETOWN,MIDDLESEX,CT,MIDDLETOWN,NaN,1,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,0,1,"MIDDLETOWN, CT\n(41.544654, -72.651713)",2
4,19,16-0889,12/20/2016 12:00:00 AM,0.0,32.0,Male,White,WINDHAM,WINDHAM,CT,WINDHAM,NaN,1,0,1,0,0,0,0,0,1,0,0,0,0,0,NaN,0,1,"Windham, CT\n(41.699744, -72.157703)",3


In [10]:
#Use weather API to extract latitude and longitude data for each city to use for HVPLOT
#test weather API call for 1 city to make sure it works

# Set the API base URL
base_url = "http://api.openweathermap.org/data/2.5/weather?"


# Create endpoint URL with each city, testing it with 2nd city in list
city_url = base_url + "appid=" + weather_api_key + "&q=" + "Middletown"
response = requests.get(city_url).json()
pprint(response)

{'base': 'stations',
 'clouds': {'all': 40},
 'cod': 200,
 'coord': {'lat': 39.5151, 'lon': -84.3983},
 'dt': 1691877953,
 'id': 4518264,
 'main': {'feels_like': 307.25,
          'humidity': 57,
          'pressure': 1007,
          'temp': 304.14,
          'temp_max': 305.96,
          'temp_min': 302.1},
 'name': 'Middletown',
 'sys': {'country': 'US',
         'id': 5016,
         'sunrise': 1691837227,
         'sunset': 1691887102,
         'type': 1},
 'timezone': -14400,
 'visibility': 10000,
 'weather': [{'description': 'scattered clouds',
              'icon': '03d',
              'id': 802,
              'main': 'Clouds'}],
 'wind': {'deg': 250, 'gust': 12.86, 'speed': 11.32}}


In [11]:
#Grouping by city to condense rows and make for loop below go much faster. 3425 rows -> 207 rows
groupby_city = clean_data.groupby(["DeathCity"]).sum()


C:\Users\Mark McLaughlin\AppData\Local\Temp\ipykernel_14420\2646773853.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  groupby_city = clean_data.groupby(["DeathCity"]).sum()


In [12]:
#use for loop to loop through cities and extract latitude and longitude for each city

#define empty list to append lat and lng data into
lat = []
lng = []

#loop through city list and extract lat and lng for each city, and append them to their respective list. groupby_city.index is the city name
for city in groupby_city.index:
    print(city)
    
    #define base URL
    base_url = "http://api.openweathermap.org/data/2.5/weather?"


    # Create endpoint URL with each city, testing it with 2nd city in list
    city_url = base_url + "appid=" + weather_api_key + "&q=" + city
    response = requests.get(city_url).json()
    
    #need try/except statment if not lat/lng data is available.
    try:
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        
    #need to fill in null value in lat & lng lists if no data is there because the list length has to be the same as DF length to add the lists to the DF
    except:
        print("No coordinates available")
        #lat.append(np.nan) #putting in null values stops plot from running. Put in 0 and filter them out later
        lat.append(0)
        lng.append(0)

groupby_city["Latitude"] = lat
groupby_city["Longitude"] = lng
groupby_city

6340
AMSTON
ANDOVER
ANSONIA
ASHFORD
AVON
BAKERSVILLE
BALTIC
BANTAM
BEACON FALLS
BERLIN
BETHANY
BETHEL
BETHLEHEM
BLOOMFIELD
BOLTON
BRANFORD
BRIDGEPORT
BRIDGEWATER
BRISTOL
BROAD BROOK
BROOKFIELD
BROOKLYN
BURLINGTON
CANTERBURY
CANTON
CENTRAL VILLAGE
CHAPLIN
CHESHIRE
CHESTER
CLINTON
COLCHESTER
COLUMBIA
CORNWALL BRIDGE
COS COB
COVENTRY
CROMWELL
DANBURY
DANIELSON
DARIEN
DAYVILLE
DEEP RIVER
DERBY
DURHAM
EAST CANAAN
EAST GRANBY
EAST HADDAM
EAST HAMPTON
EAST HARTFORD
EAST HAVEN
EAST LYME
No coordinates available
EAST WINDSOR
EASTFORD
EASTON
ELLINGTON
ENFIELD
ESSEX
FAIRFIELD
FARMINGTON
FRANKLIN
GALES FERRY
GLASTONBURY
GOSHEN
GRANBY
GREENWICH
GRISWOLD
GROTON
GROTON LONG POINT
GUILFORD
HADDAM
HAMDEN
HAMPTON
HANOVER
HARTFORD
HARWINTON
HEBRON
HIGGANUM
IVORYTON
JEWETT CITY
KENT
KILLINGLY
No coordinates available
KILLINGWORTH
LEBANON
LEDYARD
LISBON
LITCHFIELD
MADISON
MANCHESTER
MANSFIELD
MARLBOROUGH
MASHANTUCKET
No coordinates available
MERIDEN
MIDDLEBURY
MIDDLEFIELD
MIDDLETOWN
MILFORD
MONROE
MONTVILL

,Unnamed: 0,DateType,Age,Heroin,Cocaine,Fentanyl,Fentanyl_Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,OpiateNOS,Total Drug Deaths,Latitude,Longitude
DeathCity,,,,,,,,,,,,,,,,,,,,,
6340,4848,1.0,51.0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,3,24.5954,88.2705
AMSTON,4558,1.0,32.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,41.6254,-72.3431
ANDOVER,6936,1.0,91.0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,4,51.2083,-1.4825
ANSONIA,51107,10.0,686.0,4,4,6,1,5,0,4,0,6,4,1,0,1,0,0,36,41.3462,-73.0790
ASHFORD,14060,5.0,181.0,3,4,3,1,0,0,3,0,2,0,1,0,0,0,0,17,51.1459,0.8728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WINSTED,16254,6.0,207.0,5,1,3,0,0,0,1,0,1,2,1,0,1,0,1,15,41.9212,-73.0601
WOLCOTT,2290,1.0,83.0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,3,41.6023,-72.9868
WOODBRIDGE,6434,2.0,107.0,2,0,1,0,1,0,0,0,2,0,0,1,0,0,0,7,40.5576,-74.2846


In [13]:
# Configure the map plot
map1 = groupby_city.hvplot.points("Longitude","Latitude", geo=True, tiles="OSM", 
                                  frame_width = 700, frame_height = 500,
                                  size = "Total Drug Deaths")
map1

C:\Users\Mark McLaughlin\anaconda3\lib\site-packages\geoviews\operation\__init__.py:14: HoloviewsDeprecationWarning: 'ResamplingOperation' is deprecated and will be removed in version 1.18, use 'ResampleOperation2D' instead.
  from holoviews.operation.datashader import (


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Total Drug Deaths)

In [14]:
#filter out areas that are not Connecticut.
groupby_city["Total Drug Deaths"].sum() #there are 7857 total deaths. Use as reference to see how many are cut out

#filter out cities outside this region. Weather.py picked up some cities with the same name in another part of the world
groupby_city_filtered = groupby_city.loc[(groupby_city["Latitude"]>40) & 
                                         (groupby_city["Latitude"]<43) & 
                                         (groupby_city["Longitude"]>-80)]

groupby_city_filtered["Total Drug Deaths"].sum() #narrowed it from 7857 cities down to 6080 cities
groupby_city_filtered.reset_index()

,DeathCity,Unnamed: 0,DateType,Age,Heroin,Cocaine,Fentanyl,Fentanyl_Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,OpiateNOS,Total Drug Deaths,Latitude,Longitude
0,AMSTON,4558,1.0,32.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,41.6254,-72.3431
1,ANSONIA,51107,10.0,686.0,4,4,6,1,5,0,4,0,6,4,1,0,1,0,0,36,41.3462,-73.0790
2,BALTIC,1433,1.0,79.0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,4,41.6170,-72.0845
3,BEACON FALLS,4804,2.0,147.0,2,0,2,1,0,0,0,0,2,0,0,0,0,0,0,7,41.4429,-73.0626
4,BETHEL,14512,2.0,211.0,2,2,5,0,1,1,0,0,2,0,0,0,0,0,0,13,41.3712,-73.4140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,WINDHAM,16963,4.0,302.0,6,2,4,0,1,0,0,0,1,0,0,0,1,0,0,15,41.8334,-72.0001
100,WINDSOR LOCKS,44867,11.0,562.0,5,2,8,0,0,0,4,0,4,1,0,0,0,0,0,24,41.9293,-72.6273
101,WINSTED,16254,6.0,207.0,5,1,3,0,0,0,1,0,1,2,1,0,1,0,1,15,41.9212,-73.0601
102,WOLCOTT,2290,1.0,83.0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,3,41.6023,-72.9868


In [15]:
#need to copy death city column to color points by city and lebel them because DeathCity is the index and I can't figure out how to color by index
groupby_city_filtered["DeathCity"] = groupby_city_filtered.index

C:\Users\Mark McLaughlin\AppData\Local\Temp\ipykernel_14420\381661874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groupby_city_filtered["DeathCity"] = groupby_city_filtered.index


In [20]:
map2 = groupby_city_filtered.hvplot.points("Longitude","Latitude", geo=True, tiles="OSM", 
                                  frame_width = 700, frame_height = 500,
                                  size = "Total Drug Deaths", 
                                  color = "DeathCity")
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (DeathCity,Total Drug Deaths)